# Lab: Machine learning  
*Using the cardiovascular disease dataset.*

## EDA

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
df = pd.read_csv("../data/cardio.csv", sep=";")
df = df.drop(columns=["id"])

In [ ]:
sns.countplot(data=df, x="cardio");

In [ ]:
plt.pie(df["cholesterol"].value_counts(), labels=["normal", "above normal", "well above normal"], autopct="%1.1f%%", startangle=178);

In [ ]:
sns.histplot(data=df, x="age", hue="cardio");

In [ ]:
sns.countplot(data=df, x="smoke", hue="cardio");

In [ ]:
sns.histplot(data=df, x="weight", hue="gender", bins=75);

In [ ]:
sns.histplot(data=df, x="height", hue="gender", bins=60);

In [ ]:
sns.countplot(data=df, x="cardio", hue="gender");

## Feature engineering
### BMI

In [ ]:
df["bmi"] = df["weight"] / (df["height"] / 100) ** 2
df["bmi"] = df["bmi"].round(1)
df = df[["age", "gender", "height", "weight", "bmi", "ap_hi", "ap_lo", "cholesterol", "gluc", "smoke", "alco", "active", "cardio"]]
df = df[(df["bmi"] > 16) & (df["bmi"] <= 40)]

Då labbinstruktionerna hänvisar till Wikipedia används de gränsvärden angivna där. Ursprunglig källa anges vara WHO och hänvisar till att människor med värden under 16 är "svårt tunna" medan de med värden över 40 lider av "fetma klass 3". 

In [ ]:
bmi_labels = ["underweight", "normal", "overweight", "obese cl1", "obese cl2", "obese cl3"]
df["bmi_cat"] = pd.cut(df["bmi"], bins=[16, 18.5, 25, 30, 35, 40, float("inf")], labels=bmi_labels)
df = df[["age", "gender", "height", "weight", "bmi", "bmi_cat", "ap_hi", "ap_lo", "cholesterol", "gluc", "smoke", "alco", "active", "cardio"]]

### blodtryck

In [ ]:
df = df[(df["ap_hi"] >= 0) & (df["ap_hi"] <= 250) &
        (df["ap_lo"] >= 0) & (df["ap_lo"] <= 200)]
df.describe()

In [ ]:
fig, ax = plt.subplots(1, 2)
sns.boxplot(data=df, y="ap_hi", ax=ax[0])
sns.boxplot(data=df, y="ap_lo", ax=ax[1])
plt.show()

Eftersom många outliers fortfarande är närvarande i datan används nedan IQR-metoden för att rensa fler outliers. 

In [ ]:
def clear_outliers(df):    
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    df = df[(df >= lower) & (df <= upper)]
    return df

df["ap_hi"] = clear_outliers(df["ap_hi"])
df["ap_lo"] = clear_outliers(df["ap_lo"])
df = df.dropna(how="any")

In [ ]:
fig, ax = plt.subplots(1, 2)
sns.boxplot(data=df, y="ap_hi", ax=ax[0])
sns.boxplot(data=df, y="ap_lo", ax=ax[1])
plt.show()

In [ ]:
bp_conditions = [
    (df["ap_hi"] <= 90) | (df["ap_lo"] <= 60), # with outlier I need | instead of & otherwise I get unknown values
    ((df["ap_hi"] >= 90) & (df["ap_hi"] < 120)) & (df["ap_lo"] < 80),
    ((df["ap_hi"] >= 120) & (df["ap_hi"] < 130)) & (df["ap_lo"] < 80),
    ((df["ap_hi"] >= 130) & (df["ap_hi"] < 140)) | ((df["ap_lo"] >= 80) & (df["ap_lo"] < 90)),
    ((df["ap_hi"] >= 140) & (df["ap_hi"] < 180)) | ((df["ap_lo"] >= 90) & (df["ap_lo"] < 120)),
    (df["ap_hi"] >= 180) | (df["ap_lo"] >= 120)
]
bp_labels = [
    "hypotension", 
    "normal", 
    "elevated", 
    "hypertension st1", 
    "hypertension st2", 
    "hypertension crisis"
]

df["ap_cat"] = np.select(bp_conditions, bp_labels, default="unknown")
df = df[["age", "gender", "height", "weight", "bmi", "bmi_cat", "ap_hi", "ap_lo", "ap_cat", "cholesterol", "gluc", "smoke", "alco", "active", "cardio"]]
df

In [ ]:
df["ap_cat"].value_counts()

Då labbinstruktionerna hänvisar till Healthline används de gränsvärden angivna där. Det framgår även en källhänvisning till brittiska NHS som anger gränsvärden även för lågt blodtryck vilket implementerats här likaså. 

## Visualiseringar
### Andel sjukdomar

In [ ]:
chol_gluc_labels = ["normal", "above normal", "well above normal"]
binary_labels = ["no", "yes"]

In [ ]:
# TODO: barplots instead? 

fig, ax = plt.subplots(3, 2, figsize=(20, 20))

sns.countplot(data=df, x="bmi_cat", order=bmi_labels, hue="cardio", ax=ax[0, 0])
ax[0, 0].tick_params(axis="x", rotation=45)
ax[0, 0].set_title("Body Mass Index\n", fontweight="bold")
ax[0, 0].set_xlabel("")
ax[0, 0].set_ylabel("")

sns.countplot(data=df, x="ap_cat", order=bp_labels, hue="cardio", ax=ax[0, 1])
ax[0, 1].tick_params(axis="x", rotation=45)
ax[0, 1].set_title("Blood Pressure\n", fontweight="bold")
ax[0, 1].set_xlabel("")
ax[0, 1].set_ylabel("")

sns.countplot(data=df, x=(df["age"] // 365.25).astype(int), hue="cardio", ax=ax[1, 0])
ax[1, 0].tick_params(axis="x", rotation=45)
ax[1, 0].set_title("Age\n", fontweight="bold")
ax[1, 0].set_xlabel("")
ax[1, 0].set_ylabel("")

sns.countplot(data=df, x="cholesterol", hue="cardio", ax=ax[1, 1])
ax[1, 1].tick_params(axis="x", rotation=45)
ax[1, 1].set_xticks([0, 1, 2])
ax[1, 1].set_xticklabels(chol_gluc_labels)
ax[1, 1].set_title("Cholesterol Levels\n", fontweight="bold")
ax[1, 1].set_xlabel("")
ax[1, 1].set_ylabel("")

sns.countplot(data=df, x="gluc", hue="cardio", ax=ax[2, 0])
ax[2, 0].tick_params(axis="x", rotation=45)
ax[2, 0].set_xticks([0, 1, 2])
ax[2, 0].set_xticklabels(chol_gluc_labels)
ax[2, 0].set_title("Glucose Levels\n", fontweight="bold")
ax[2, 0].set_xlabel("")
ax[2, 0].set_ylabel("")

sns.countplot(data=df, x="active", hue="cardio", ax=ax[2, 1])
ax[2, 1].tick_params(axis="x", rotation=45)
ax[2, 1].set_xticks([0, 1])
ax[2, 1].set_xticklabels(binary_labels)
ax[2, 1].set_title("Physically Active\n", fontweight="bold")
ax[2, 1].set_xlabel("")
ax[2, 1].set_ylabel("")

# fig.delaxes(ax[2, 1])  # Remove the empty subplot

fig.suptitle("\nVisualisation of CVD risk factors", fontweight="bold", fontsize=20)
plt.subplots_adjust(hspace=0.4)
plt.show()

### Korrelation

In [ ]:
# TODO: change all values to numeric
numerical_columns = df.describe().columns.to_list()
corr = df[numerical_columns].corr()
plt.figure(figsize=(9, 7))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", center=0, square=True, linewidths=0.5)
plt.show()

BMI och vikt samt systoliskt och diastoliskt blodtryck kan ignoreras pga deras uppenbara relationer. Detsamma skulle kunna sägas för glukos och kolesterol.  

Mer intressanta observationer för detta dataset är att rökning har en viss relation till alkoholkonsumption men även kön.  

För syftet med labben är den sista raden den enda viktiga, korrelationer med vår responsvariabel. Då BMI har en högre korrelation än de separata variablerna för höjd och vikt vore det intressant att veta hur vår nuvarande kategoriska variabel för blodtryck påverkas. Andra kolumner av värde är ålder och kolesterol. 

## Dataset

In [ ]:
df_a = df.copy().drop(columns=["height", "weight", "bmi", "ap_hi", "ap_lo"])
df_b = df.copy().drop(columns=["height", "weight", "bmi_cat", "ap_cat"])

df_a = pd.get_dummies(df_a, columns=["gender", "bmi_cat", "ap_cat"])
df_b = pd.get_dummies(df_b, columns=["gender"])

## Modeller

Välj fem modeller:  
- Logistic Regression  
- Support Vector Machines  
- Stochastic Gradient Descent  
- K-Nearest Neighbour  
- Random Forests  

In [ ]:
def splits(df):

    X_train, X_temp, y_train, y_temp = train_test_split(
        df.drop("cardio", axis=1), 
        df["cardio"], 
        test_size=0.3, 
        random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, 
        y_temp, 
        test_size=0.5, 
        random_state=42
    )
    return X_train, X_val, X_test, y_train, y_val, y_test


X_train_a, X_val_a, X_test_a, y_train_a, y_val_a, y_test_a = splits(df_a)
X_train_b, X_val_b, X_test_b, y_train_b, y_val_b, y_test_b = splits(df_b)

In [ ]:
def standardiser(X_train, X_val, X_test):

    scaler = StandardScaler()
    X_train_std = scaler.fit_transform(X_train)
    X_val_std = scaler.transform(X_val)
    X_test_std = scaler.transform(X_test)
    
    return X_train_std, X_val_std, X_test_std


def normaliser(X_train, X_val, X_test):
    
    scaler = MinMaxScaler()
    X_train_norm = scaler.fit_transform(X_train)
    X_val_norm = scaler.transform(X_val)
    X_test_norm = scaler.transform(X_test)
    
    return X_train_norm, X_val_norm, X_test_norm


X_train_a_std, X_val_a_std, X_test_a_std = standardiser(X_train_a, X_val_a, X_test_a)
X_train_b_std, X_val_b_std, X_test_b_std = standardiser(X_train_b, X_val_b, X_test_b)
X_train_a_norm, X_val_a_norm, X_test_a_norm = normaliser(X_train_a, X_val_a, X_test_a)
X_train_b_norm, X_val_b_norm, X_test_b_norm = normaliser(X_train_b, X_val_b, X_test_b)

### Martins tips: spara resultat från modeller i CSV-filer så 